Below is a function that will create a baseline neural network for the iris classication
problem. It creates a simple fully connected network with one hidden layer that contains 8
neurons. The hidden layer uses a rectifier activation function which is a good practice. Because
we used a one hot encoding for our iris dataset, the output layer must create 3 output values,
one for each class. The output value with the largest value will be taken as the class predicted
by the model. The network topology of this simple one-layer neural network can be summarized
as: <p>
4 inputs -> 8 hidden nodes -> 3 outputs <p>
Softmax activation function in the output layer.

Check mis redes en tipo de arquitectura con: https://playground.tensorflow.org/

In [21]:
import numpy
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [22]:
""" to ensure that the results we achieve from this model can be achieved again precisely. It ensures
that the stochastic process of training a neural network model can be reproduced """
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [23]:
from sklearn.datasets import load_iris

iris_ds = load_iris()
X = iris_ds.data
# convert integers to dummy variables (i.e. one hot encoded)
Y_one_hot_encoded = np_utils.to_categorical(iris_ds.target)
Y_one_hot_encoded[:4]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [24]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [25]:
""" to ensure that the results we achieve from this model can be achieved again precisely. It ensures
that the stochastic process of training a neural network model can be reproduced """
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

### Pasamos a: <p>
   * construir el estimador a utilizar en la estrategia de validación cruzada
   * dividir el dataset en 10 folds
   * ejecutar el entrenamiento y validación con cada una de las 10 parejas train-validation sets

In [26]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y_one_hot_encoded, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 98.67% (2.67%)


In [28]:
results

array([1.        , 1.        , 1.        , 1.        , 0.93333334,
       1.        , 1.        , 0.93333334, 1.        , 1.        ])

### Model serialization

In [32]:
from tensorflow.keras.models import model_from_json

# entrenamos el modelo keras:
model = baseline_model()
model.fit(X, Y_one_hot_encoded, epochs=150, batch_size=10, verbose=0)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")


Saved model to disk


### Y ahora cargamos el modelo guardado

In [37]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y_one_hot_encoded, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))


ValueError: Unknown initializer: GlorotUniform

## AHORA LA IDEA SERÍA PROBAR CON MÁS MODELOS, TANTO REDES NEURONALES MÁS COMPLETAS COMO OTROS MODELOS ML Y EMPEZAR A PENSAR EN COMPARADORES DE MODELOS POR TIPOLOGÍAS DE DATASETS ETC 